### Sprawozdanie - Singular Value Decomposition
<div style="text-align: right"> Wojciech Kosztyła </div>

#### Zadanie 1 - Wyszukiwarka

In [2]:
import math
import numpy as np

1. Przygotuj duży (> 1000 elementów) zbiór dokumentów tekstowych w języku angielskim (np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór dokumentów HTML uzyskanych za pomocą Web crawlera, zbiór rozdziałów wyciętych z różnych książek)

Pobrałem 2000 recenzji "polarity_dataset_v2.0" z http://www.cs.cornell.edu/people/pabo/movie-review-data/

2. Określ słownik słów kluczowych (termów) potrzebny do wyznaczenia wektorów cech bag-of-words (indeksacja). Przykładowo zbiorem takim może być unia wszystkich słów występujących we wszystkich tekstach.

In [3]:
ilosc_dokumentow = 450

def otworz_dokument_tekstowy(numer):
    numer_string = str(numer)
    while(len(numer_string) < 4):
        numer_string = "0" + numer_string
    return open("recenzje/recenzja_" + numer_string + ".txt")

def wczytaj_dokument(i):
    plik = otworz_dokument_tekstowy(i)
    temp_slowa = plik.read().lower()
    temp_slowa = list(temp_slowa)
    dlugosc_pliku = len(temp_slowa)
    for pozycja in range(dlugosc_pliku):
        if (temp_slowa[pozycja] == "," or temp_slowa[pozycja] == "." or temp_slowa[pozycja] == ":" or temp_slowa[pozycja] == "(" or temp_slowa[pozycja] == ")" or temp_slowa[pozycja] == '"' or temp_slowa[pozycja] == "?" or temp_slowa[pozycja] == "!" or temp_slowa[pozycja] == "&" or temp_slowa[pozycja] == "-" or temp_slowa[pozycja] == "/" or temp_slowa[pozycja].isdigit()):
            temp_slowa[pozycja] = ' '
    temp_slowa = "".join(temp_slowa)
    temp_slowa = temp_slowa.split()
    return temp_slowa

def wczytaj_wszystkie_dokumenty():
    slowa = []
    last_said = 0
    for i in range(1, ilosc_dokumentow + 1):
        slowa += wczytaj_dokument(i)
        if int(100*i / ilosc_dokumentow) > last_said:   # wypisuje logi co 1%
            print("Wczytałem plik nr."+str(i))
            last_said = int(100*i / ilosc_dokumentow)
    return slowa

def tworz_slownik_slow_kluczowych(slowa):
    slownik = {}
    for slowo in slowa:
        if slowo in slownik.keys():
            slownik[slowo] += 1
        else:
            slownik[slowo] = 1
    return slownik


otworz_dokument_tekstowy(1)
wszystkie_slowa = wczytaj_wszystkie_dokumenty()
slownik = tworz_slownik_slow_kluczowych(wszystkie_slowa)

print(slownik)


Wczytałem plik nr.5
Wczytałem plik nr.9
Wczytałem plik nr.14
Wczytałem plik nr.18
Wczytałem plik nr.23
Wczytałem plik nr.27
Wczytałem plik nr.32
Wczytałem plik nr.36
Wczytałem plik nr.41
Wczytałem plik nr.45
Wczytałem plik nr.50
Wczytałem plik nr.54
Wczytałem plik nr.59
Wczytałem plik nr.63
Wczytałem plik nr.68
Wczytałem plik nr.72
Wczytałem plik nr.77
Wczytałem plik nr.81
Wczytałem plik nr.86
Wczytałem plik nr.90
Wczytałem plik nr.95
Wczytałem plik nr.99
Wczytałem plik nr.104
Wczytałem plik nr.108
Wczytałem plik nr.113
Wczytałem plik nr.117
Wczytałem plik nr.122
Wczytałem plik nr.126
Wczytałem plik nr.131
Wczytałem plik nr.135
Wczytałem plik nr.140
Wczytałem plik nr.144
Wczytałem plik nr.149
Wczytałem plik nr.153
Wczytałem plik nr.158
Wczytałem plik nr.162
Wczytałem plik nr.167
Wczytałem plik nr.171
Wczytałem plik nr.176
Wczytałem plik nr.180
Wczytałem plik nr.185
Wczytałem plik nr.189
Wczytałem plik nr.194
Wczytałem plik nr.198
Wczytałem plik nr.203
Wczytałem plik nr.207
Wczytałem pl

3. Dla każdego dokumentu j wyznacz wektor cech bag-of-words dj zawierający częstości występowania poszczególnych słów (termów) w tekście.

Funkcje tworz_slownik_slow_kluczowych() i wczytaj_dokument() to implementują. Jedynie wystarczy podzielić wartości <ilość wystąpień> na prawdopodobieństwo.

In [4]:
def tworz_bag_of_words_z_slownika(slownik):
    suma_ilosci = 0.0
    for ilosc, slowo in enumerate(slownik):
        suma_ilosci += int(ilosc)
    for ilosc, slowo in enumerate(slownik):
        slownik[slowo] = ilosc / suma_ilosci
    return slownik

def tworz_bag_of_words_z_dokumentu(n):
    nieprzebrane_slowa = wczytaj_dokument(n)
    slownik_ilosci_wystepowania = tworz_slownik_slow_kluczowych(nieprzebrane_slowa)
    bag_of_words = tworz_bag_of_words_z_slownika(slownik_ilosci_wystepowania)
    return bag_of_words


#tworz_bag_of_words_z_slownika(slownik)

4. Zbuduj rzadką macierz wektorów cech term-by-document matrix w której wektory cech ułożone są kolumnowo $A_{m×n} = [d1|d2|...|dn]$ (m jest liczbą termów w słowniku, a n liczbą dokumentów)

In [5]:
wszystkie_worki = []
for i in range(1,ilosc_dokumentow+1):
    wszystkie_worki.append(tworz_bag_of_words_z_dokumentu(i))




def stworz_macierz_term_by_document(slownik, dokumenty):
    macierz = np.zeros((len(slownik), len(dokumenty)))
    i = 0
    for licznik, klucz in enumerate(slownik):
        j = 0
        for dokument in dokumenty:
            if klucz in dokument:
                macierz[i, j] = dokument[klucz]
            j += 1
        i += 1
    return macierz

rzadka_macierz = stworz_macierz_term_by_document(slownik, wszystkie_worki)
#print(rzadka_macierz)

5. Przetwórz wstępnie otrzymany zbiór danych mnożąc elementy bag-of-words przez inverse document frequency. Operacja ta pozwoli na redukcję znaczenia często występujących słów.

$IDF(w) = log\frac{N}{n_{w}}$

gdzie $n_{w}$ jest liczbą dokumentów, w których występuje słowo w, a $N$ jest całkowitą liczbą dokumentów.

In [6]:
def zmien_z_IDF(macierz):
    m = len(macierz)
    n = len(macierz[0])
    for y in range(m):
        ilosc_wystapien = 0
        for x in range(n):
            if macierz[y,x] > 0:
                ilosc_wystapien += 1
        if ilosc_wystapien == 0:
            continue
        idf = math.log((1.0 * n / ilosc_wystapien),10)
        for x in range(n):
            if macierz[y,x] > 0:
                macierz[y,x] *= idf
    return macierz


rzadka_macierz_po_IDF = zmien_z_IDF(rzadka_macierz)

6. Napisz program pozwalający na wprowadzenie zapytania (w postaci sekwencji słów) przekształcanego następnie do reprezentacji wektorowej q (bag-of-words). Program ma zwrócić k dokumentów najbardziej zbliżonych do podanego zapytania q. Użyj korelacji między wektorami jako miary podobieństwa

$ cos(θ)_{j} = \frac{q^{T}d_{j}}{||q||*||d_{j}||}=\frac{q^{T}Ae_{j}}{||q||*||Ae_{j}||}$

7. Zastosuj normalizację wektorów cech dj i wektora q, tak aby miały one długość 1. Użyj zmodyﬁkowanej miary podobieństwa otrzymując

$|q^{T}A|= [|cos(θ)_{1}|,|cos(θ)_{2}|,...,|cos(θ)_{n}|]$

In [7]:
query_bag_of_words = tworz_bag_of_words_z_dokumentu(0)  # specjalnie przygotowany dokument z fragmentem jednego z 2000 plików
query_document = stworz_macierz_term_by_document(slownik, [query_bag_of_words])

def query_podobienstwa(bag_of_words_query, k, macierz_cech):
    q = np.array(bag_of_words_query)
    q_norm = np.linalg.norm(q)

    wyniki_podobienstwa_temp = []
    for j in range(ilosc_dokumentow):
        temp_gora = q.T @ macierz_cech[:,j]
        temp_dol = q_norm * np.linalg.norm(macierz_cech[:,j])
        temp_wynik = temp_gora / temp_dol
        wyniki_podobienstwa_temp.append(temp_wynik[0])

    wyniki_podobienstwa_temp /= np.linalg.norm(wyniki_podobienstwa_temp)    # normalizacja

    wyniki_podobienstwa = []
    wyniki_indexy = []
    for _ in range(k):
        temp_max = 0
        temp_index_max = 0
        j = 0
        for prawdopodobienstwo in wyniki_podobienstwa_temp:
            if prawdopodobienstwo>temp_max:
                temp_max = prawdopodobienstwo
                temp_index_max = j
            j += 1
        wyniki_podobienstwa.append(temp_max)
        wyniki_indexy.append(temp_index_max)
        wyniki_podobienstwa_temp[temp_index_max] = 0

    #wyniki_podobienstwa = []
    #for array in wyniki_podobienstwa_temp:
     #   wyniki_podobienstwa.append()
    return wyniki_podobienstwa, wyniki_indexy

wyniki_prawd, wyniki_indexy = query_podobienstwa(query_document, 5, rzadka_macierz_po_IDF)
print(wyniki_prawd)
print(wyniki_indexy)

[0.8535059714171003, 0.079468172175065, 0.06799495398648463, 0.06690324126241341, 0.05839201400666876]
[419, 153, 383, 321, 45]


Jak widać, największe podobieństwo do pliku recenzja_0000.txt ma recenzja_0420.txt (czyli w indeksowaniu od 0 -> 419),
gdyż "0000" to wycięty kawałek "0420".




8. W celu usunięcia szumu z macierzy A zastosuj SVD i low rank approximation otrzymując

![text](Adnotacja 2020-05-30 140957.png)

oraz nową miarę podobieństwa

$cos(\phi)_{j} == \frac{q^{T}A_{k}e_{j}}{||q||*||A_{k}e_{j}||}$

In [ ]:
def svd_i_low_rank_approx(macierz_przed, k):
    u, s, vh = np.linalg.svd(macierz_przed)
    s = np.diag(s)
    for i in range(k+1, len(s)):
        s[i,i] = 0
    return u @ s @ vh

macierz_A = svd_i_low_rank_approx(rzadka_macierz_po_IDF, 50)

wyniki_prawd, wyniki_indexy = query_podobienstwa(query_document, 5, macierz_A)
print(wyniki_prawd)
print(wyniki_indexy)

9. Porównaj działanie programu bez usuwania szumu i z usuwaniem szumu. Dla jakiej wartości k wyniki wyszukiwania są najlepsze (subiektywnie). Zbadaj wpływ przekształcenia IDF na wyniki wyszukiwania.

Dla plików przygotowanych przeze mnie, dla k w granicach ~70-80 dawało wyniki niemalże idealne.
Przekształcenie IDF sprawiło, że słowa, które nie pojawiały się tak często miały mniejszą wagę w przeszukiwaniu - dzięki temu jeden z wyników zawsze się wyróżniał.